### Train a defined LSTM using prepsocessed data

Read in one or several json files. Use them to train a specified LSTM.

It is expected, that the read in data is already preprocessed, no pipeline is needed, but the 

goal values are weighted depending on the amount of samples within the respective bin.

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
 
import tensorflow as tf
import keras as ks
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.models import load_model

from keras.callbacks import EarlyStopping, CSVLogger, TerminateOnNaN, ModelCheckpoint
import json

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA


In [ ]:
from weightedValues import weightValues

In [ ]:
def OpenJsontoArr(path):
    file = open(path)
    x_3d = json.load(file)
    file.close()
    x_3d = np.asarray(x_3d)
    return x_3d

In [ ]:
def get_lstm_model(layers, dropout, activation, input_shape, loss):
    
    model = Sequential()
    model.add(Input(input_shape))
    
    for i in range(len(layers)):
        if i == len(layers)-1:
            model.add(LSTM(layers[i], stateful = False, dropout=dropout, activation = activation, return_sequences=False))
        else:
            model.add(LSTM(layers[i], stateful = False, dropout=dropout, activation = activation, return_sequences=True))
        #model.add(Dropout(dropout))

    model.add(Dense(1, activation="relu"))
    # compile the model
    model.compile(
        loss= loss,
        #metrics=[ "max_loss"],
        optimizer = "Adam")
    # return compiled model

    print(model.layers)
    return model

In [ ]:
"""Custom loss function to reduce the absolute error of the individual sample as well as the 
maximum error"""
def customLoss(y_true, y_pred):
    weight = 1.5
    difference = tf.abs(y_true - y_pred)
    exponent = tf.exp(tf.multiply(weight, difference))
    weighted_muls = tf.multiply(difference, exponent)
    boltzmann_op = tf.reduce_sum(weighted_muls) / tf.reduce_sum(exponent)
    loss = tf.add(boltzmann_op, tf.losses.mean_absolute_error(y_true, y_pred))
    return loss

In [ ]:
def preprocessData(x_train, x_test, y_train, y_test):
    x_trainAll = np.array([]).reshape(0, 40, 6)
    y_trainAll = np.array([]).reshape(0,)
    x_testAll = np.array([]).reshape(0, 40, 6)
    y_testAll = np.array([]).reshape(0,)

    min_seq_length = 100000

    for i in range(len(x_train)):
        x_trainfile = OpenJsontoArr(x_train[i])

        if len(x_trainfile) < min_seq_length:
            min_seq_length = len(x_trainfile)

        x_testfile = OpenJsontoArr(x_test[i])
        x_trainfile = np.nan_to_num(x_trainfile)
        x_testfile = np.nan_to_num(x_testfile)
        y_trainfile = OpenJsontoArr(y_train[i])
        y_testfile = OpenJsontoArr(y_test[i])
        y_trainfile = y_trainfile.flatten()
        y_testfile = y_testfile.flatten()
        x_trainAll = np.concatenate((x_trainAll, x_trainfile))
        x_testAll = np.concatenate((x_testAll, x_testfile))
        y_trainAll = np.concatenate((y_trainAll, y_trainfile))
        y_testAll = np.concatenate((y_testAll, y_testfile))

    df_y_train = pd.DataFrame({"y": y_trainAll})
    df_weights_train = weightValues(df_y_train, weightMin=1, weightMax=1, namey="y", nameWeights="weights")
    df_y_test = pd.DataFrame({"y": y_testAll})
    df_weights_test = weightValues(df_y_test, weightMin=1, weightMax=1, namey="y", nameWeights="weights")

    y_testAll = np.array(df_weights_test)
    y_testAll = y_testAll[:, :2]
    y_trainAll = np.array(df_weights_train)
    y_trainAll = y_trainAll[:, :2]

    return x_trainAll, x_testAll, y_trainAll, y_testAll, min_seq_length

In [ ]:
earlyStop = EarlyStopping(monitor='val_loss', patience=300, verbose =1, mode = "auto")
#csvLogger = CSVLogger('X:\\KI Praktikum\\csvLoggerCustomLoss.xlsx')
stopNaN = TerminateOnNaN()

In [ ]:
'''Function to plot the prediction compared with the real y-values'''

def plotPredError(y_pred, y_true):
    scatter_mode = 'lines'

    fig= make_subplots(rows=1, cols=1, shared_xaxes= True, print_grid= True)

    fig.add_trace(go.Scatter(y= y_pred.flatten(), name= 'prediction', mode= scatter_mode), row= 1, col= 1)
    fig.add_trace(go.Scatter(y= y_true.flatten(), name= 'trace', mode= scatter_mode), row= 1, col= 1)
    fig.update_yaxes(title_text= 'y-value', row= 1, col= 1)

    fig.update_layout(height=600, width=1200, title_text="Prediction error")
    fig.show()

In [ ]:
def trainLSTM(x_train, x_test, y_train, y_test, window, modelpath, batch_size):
    #i = 0
    #summary = np.array(len(losses))
    #for loss in losses:
    checkpoint = ModelCheckpoint(modelpath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    lstmmodel = get_lstm_model(layers=[5], dropout=0, activation = "tanh", input_shape = (window, 6), loss = tf.keras.losses.MeanSquaredError())

    summary = lstmmodel.fit(x=x_train, y=y_train[:,0], validation_data=(x_test, y_test[:,0]), sample_weight = y_train[:,1], 
                            epochs = 1000, batch_size = batch_size, callbacks=[earlyStop, stopNaN, checkpoint], shuffle = False, verbose = 0)

    y_pred_test = lstmmodel.predict(x_test)
    y_pred_train = lstmmodel.predict(x_train)

    diff_test = y_test[:,0] - y_pred_test.flatten()
    error_avg_test = np.mean(abs(diff_test))
    error_max_test = np.max(abs(diff_test))

    diff_train = y_train[:,0] - y_pred_train.flatten()
    error_avg_train = np.mean(abs(diff_train))
    error_max_train = np.max(abs(diff_train))

    #mean_absolute = tf.keras.losses.mean_absolute_error(y_cv[:,0], y_pred.flatten())
    #print("mean absolute: ", mean_absolute)
    print("Validation scores: ")
    print("max error: ", error_max_test)
    print("mean abs error: ", error_avg_test)
    print("Error over the training data: ")
    print("max error: ", error_max_train)
    print("mean abs error: ", error_avg_train)
    #i+=1

    plotPredError(y_pred_test, y_test[:,0])
    plotPredError(y_pred_train, y_train[:,0])
    
    return summary

In [ ]:
def plotTrainValLoss(summary):
    scatter_mode = "lines"
    train_loss = summary.history["loss"]
    validation_loss = summary.history["val_loss"]

    fig= make_subplots(rows=1, cols=1, shared_xaxes= True, print_grid= True)

    fig.add_trace(go.Scatter( y= train_loss, name= 'training loss', mode= scatter_mode), row= 1, col= 1)
    fig.add_trace(go.Scatter( y= validation_loss, name= 'validation loss', mode= scatter_mode), row= 1, col= 1)
    fig.update_yaxes(title_text= 'loss', row= 1, col= 1)

    fig.update_layout(height=600, width=1200, title_text="Training loss curve")
    fig.show()

In [ ]:
# x_train = glob.glob(os.path.join("X:\\RotorTempKI\\train", "*_x.json"))
# y_train = glob.glob(os.path.join("X:\\RotorTempKI\\train", "*_y.json"))
# x_test = glob.glob(os.path.join("X:\\RotorTempKI\\validation", "*_x.json"))
# y_test = glob.glob(os.path.join("X:\\RotorTempKI\\validation", "*_y.json"))
# window_size = [50, 60, 70, 50, 60, 70, 50, 60, 70]

In [ ]:
# for i in range(len(x_train)):
#     print(x_train[i])
#     x_traini, x_testi, y_traini, y_testi = preprocessData(x_train[i], x_test[i], y_train[i], y_test[i])
#     summary = trainLSTM(x_traini, x_testi, y_traini, y_testi, window_size[i])
#     plotTrainValLoss(summary)

In [ ]:
x_train = glob.glob(os.path.join("C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\LSTMjsonTrain", "*_x.json"))
y_train = glob.glob(os.path.join("C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\LSTMjsonTrain", "*_y.json"))
x_test = glob.glob(os.path.join("C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\LSTMjsonVal", "*_x.json"))
y_test = glob.glob(os.path.join("C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\LSTMjsonVal", "*_y.json"))
window_size = [40]

In [ ]:
# Preprocess data:
x_traini, x_testi, y_traini, y_testi, min_seq_length = preprocessData(x_train, x_test, y_train, y_test)

In [ ]:

filepath = "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\modelle\\" + "14-09-23_s1_w40" +".h5"
print(filepath)

x_train=x_traini
x_test=x_testi
y_train=y_traini
y_test=y_testi
window=window_size[0]
modelpath = filepath
batch_size = min_seq_length

checkpoint = ModelCheckpoint(modelpath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
lstmmodel = get_lstm_model(layers=[10], dropout=0, activation = "elu", input_shape = (window, 6), loss = tf.keras.losses.MeanSquaredError())

summary = lstmmodel.fit(x=x_train, y=y_train[:,0], validation_data=(x_test, y_test[:,0]), sample_weight = y_train[:,1], 
                        epochs = 2000, batch_size = batch_size, callbacks=[earlyStop, checkpoint], shuffle = False, verbose = 1)

y_pred_test = lstmmodel.predict(x_test)
y_pred_train = lstmmodel.predict(x_train)

diff_test = y_test[:,0] - y_pred_test.flatten()
error_avg_test = np.mean(abs(diff_test))
error_max_test = np.max(abs(diff_test))

diff_train = y_train[:,0] - y_pred_train.flatten()
error_avg_train = np.mean(abs(diff_train))
error_max_train = np.max(abs(diff_train))

#mean_absolute = tf.keras.losses.mean_absolute_error(y_cv[:,0], y_pred.flatten())
#print("mean absolute: ", mean_absolute)
print("Validation scores: ")
print("max error: ", error_max_test)
print("mean abs error: ", error_avg_test)
print("Error over the training data: ")
print("max error: ", error_max_train)
print("mean abs error: ", error_avg_train)
#i+=1

plotPredError(y_pred_test, y_test[:,0])
plotPredError(y_pred_train, y_train[:,0])


plotTrainValLoss(summary)

In [ ]:
#for i in range(len(x_train)):
filepath = "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\modelle\\" + "14-09-23_s1_w40" +".h5"
print(filepath)
summary = trainLSTM(x_train=x_traini, x_test=x_testi, y_train=y_traini, y_test=y_testi, window=window_size[0], modelpath = filepath, batch_size = min_seq_length)
plotTrainValLoss(summary)

In [ ]:
# modell = load_model(filepath)
# #summary = modell.fit(x_)
# y_pred = modell.predict(x_testi)
# y_diff = y_testi - y_pred
# avg = np.mean(abs(y_diff))
# max = np.max(abs(y_diff))
# print("avg. error: " + str(avg))
# print("max. error: " + str(max))
# plotPredError(y_pred, y_testi[:,0])

# y_pred_train = modell.predict(x_traini)
# plotPredError(y_pred_train, y_traini[:,0])